# KPI attaquants — Notebook prêt à l'emploi
- Source CSV: `ressources/normalized_data/assembled_data_FW_normalized.csv`
- Bibliothèques: pandas, numpy, matplotlib

Ce notebook charge les données, documente les colonnes, calcule des KPI utiles pour les attaquants et exporte dans un csv

In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CSV_PATH = r"../normalized_data/assembled_data_FW_normalized.csv"

df = pd.read_csv(CSV_PATH)


In [5]:

def safe_div(a, b):
    b = b.replace(0, np.nan)
    return a / b

def per90(col, df):
    if f"{col}_per_90" in df:
        return df[f"{col}_per_90"]
    if "90s" in df:
        return df[col] / df["90s"]
    if "Min" in df:
        return df[col] * 90 / df["Min"].replace(0, np.nan)
    raise KeyError("Ni 90s ni Min disponibles")


In [6]:

# Dictionnaire des colonnes (auto-généré, heuristique)
_dict = [{'colonne': 'Rk', 'unite': 'compte', 'categorie': 'meta', 'origine': 'FBref', 'signification': 'rang FBref', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'Player', 'unite': 'texte', 'categorie': 'meta', 'origine': 'FBref', 'signification': 'nom du joueur', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'Nation', 'unite': 'texte', 'categorie': 'meta', 'origine': 'FBref', 'signification': 'nationalité (FIFA+code)', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'Pos', 'unite': 'texte', 'categorie': 'meta', 'origine': 'FBref', 'signification': 'poste(s) FBref', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'Squad', 'unite': 'texte', 'categorie': 'meta', 'origine': 'FBref', 'signification': 'club', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'Comp', 'unite': 'texte', 'categorie': 'meta', 'origine': 'FBref', 'signification': 'compétition', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'Age', 'unite': 'années', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': 'âge saison', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': 'Born', 'unite': 'année', 'categorie': 'meta', 'origine': 'FBref', 'signification': 'année de naissance', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'MainPos', 'unite': 'texte', 'categorie': 'meta', 'origine': 'Utilisateur', 'signification': 'poste principal (créé)', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': ''}, {'colonne': 'MP', 'unite': 'compte', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': 'matches joués', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': 'Starts', 'unite': 'compte', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': 'titularisations', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': 'Min', 'unite': 'minutes', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': 'minutes jouées', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': '90s', 'unite': 'matchs de 90 min', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': 'équivalents 90 minutes', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': 'CrdY', 'unite': 'compte', 'categorie': 'discipline', 'origine': 'FBref', 'signification': 'cartons jaunes', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': ''}, {'colonne': 'CrdR', 'unite': 'compte', 'categorie': 'discipline', 'origine': 'FBref', 'signification': 'cartons rouges', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': ''}, {'colonne': 'Gls', 'unite': 'compte', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'buts', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'Sh_shooting', 'unite': 'compte', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'tirs tentés', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'SoT', 'unite': 'compte', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'tirs cadrés', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'SoT%', 'unite': '%', 'categorie': 'tir', 'origine': 'FBref', 'signification': '% tirs cadrés / tirs', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'Sh/90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'tirs par 90', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'SoT/90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'tirs cadrés par 90', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'G/Sh', 'unite': 'ratio', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'buts par tir', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'G/SoT', 'unite': 'ratio', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'buts par tir cadré', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'Dist', 'unite': 'yards', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'distance moyenne de tir (yards)', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'FK_shooting', 'unite': 'compte', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'tirs sur CPA directs (CF)', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'xG', 'unite': 'xG', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'expected goals', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'npxG', 'unite': 'xG', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'expected goals hors pen.', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'xG_shooting', 'unite': 'xG', 'categorie': 'expected', 'origine': 'FBref', 'signification': "xG total (table 'Shooting')", 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'npxG_shooting', 'unite': 'xG', 'categorie': 'expected', 'origine': 'FBref', 'signification': "npxG total (table 'Shooting')", 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'npxG/Sh', 'unite': 'xG/tir', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'npxG par tir', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'G-xG', 'unite': 'buts - xG', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'sur/sous-performance vs xG', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'np:G-xG', 'unite': 'npG - npxG', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'sur/sous-performance vs npxG', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'PK', 'unite': 'compte', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'buts sur pénalty', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'PKatt', 'unite': 'compte', 'categorie': 'tir', 'origine': 'FBref', 'signification': 'pénaltys tentés', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': ''}, {'colonne': 'Ast', 'unite': 'compte', 'categorie': 'passes', 'origine': 'FBref', 'signification': 'passes décisives', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': ''}, {'colonne': 'KP', 'unite': 'compte', 'categorie': 'passes', 'origine': 'FBref', 'signification': 'passes clés (mènent à tir)', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': ''}, {'colonne': 'xAG', 'unite': 'xAG', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'expected assisted goals', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'xA', 'unite': 'xA', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'expected assists', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'A-xAG', 'unite': 'Ast - xAG', 'categorie': 'expected', 'origine': 'FBref', 'signification': 'sur/sous-perf assists', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': ''}, {'colonne': 'PPA', 'unite': 'compte', 'categorie': 'passes', 'origine': 'FBref', 'signification': 'passes complétées vers la surface', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': ''}, {'colonne': 'CrsPA', 'unite': 'compte', 'categorie': 'passes', 'origine': 'FBref', 'signification': 'centres vers la surface', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': ''}, {'colonne': 'Touches', 'unite': 'compte', 'categorie': 'possession', 'origine': 'FBref', 'signification': 'touches de balle', 'utilite_pour_FW': 'présence zones offensives', 'calcul_si_utilisateur': ''}, {'colonne': 'Att 3rd_possession', 'unite': 'compte', 'categorie': 'possession', 'origine': 'FBref', 'signification': 'touches dans 1/3 offensif', 'utilite_pour_FW': 'présence zones offensives', 'calcul_si_utilisateur': ''}, {'colonne': 'Att Pen', 'unite': 'compte', 'categorie': 'possession', 'origine': 'FBref', 'signification': 'touches dans la surface adverse', 'utilite_pour_FW': 'présence zones offensives', 'calcul_si_utilisateur': ''}, {'colonne': 'Att_possession', 'unite': 'compte', 'categorie': 'dribbles', 'origine': 'FBref', 'signification': 'dribbles tentés (take-ons)', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': ''}, {'colonne': 'Succ', 'unite': 'compte', 'categorie': 'dribbles', 'origine': 'FBref', 'signification': 'dribbles réussis', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': ''}, {'colonne': 'Succ%', 'unite': '%', 'categorie': 'dribbles', 'origine': 'FBref', 'signification': '% dribbles réussis', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': ''}, {'colonne': 'Tkld', 'unite': 'compte', 'categorie': 'dribbles', 'origine': 'FBref', 'signification': 'fois taclés en dribble', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': ''}, {'colonne': 'Tkld%', 'unite': '%', 'categorie': 'dribbles', 'origine': 'FBref', 'signification': '% de tentatives finies taclé', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': ''}, {'colonne': 'Carries', 'unite': 'compte', 'categorie': 'conduite', 'origine': 'FBref', 'signification': 'conduites de balle', 'utilite_pour_FW': 'analyse générale', 'calcul_si_utilisateur': ''}, {'colonne': 'PrgC', 'unite': 'compte', 'categorie': 'progression', 'origine': 'FBref', 'signification': 'conduites progressives', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': ''}, {'colonne': 'PrgDist_possession', 'unite': 'yards', 'categorie': 'progression', 'origine': 'FBref', 'signification': 'distance progressive totale (conduite)', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': ''}, {'colonne': 'CPA', 'unite': 'compte', 'categorie': 'progression', 'origine': 'FBref', 'signification': 'conduites dans la surface', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': ''}, {'colonne': 'Mis', 'unite': 'compte', 'categorie': 'pertes', 'origine': 'FBref', 'signification': 'mauvais contrôles', 'utilite_pour_FW': 'gestion du risque', 'calcul_si_utilisateur': ''}, {'colonne': 'Dis', 'unite': 'compte', 'categorie': 'pertes', 'origine': 'FBref', 'signification': 'ballons perdus (dépossédé)', 'utilite_pour_FW': 'gestion du risque', 'calcul_si_utilisateur': ''}, {'colonne': 'PrgR', 'unite': 'compte', 'categorie': 'progression', 'origine': 'FBref', 'signification': 'passes progressives reçues', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': ''}, {'colonne': 'PrgC_possession', 'unite': 'compte', 'categorie': 'progression', 'origine': 'FBref', 'signification': 'conduites progressives (alias)', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': ''}, {'colonne': '1/3_possession', 'unite': 'compte', 'categorie': 'progression', 'origine': 'FBref', 'signification': 'conduites dans le 1/3 offensif', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': ''}, {'colonne': 'Cmp', 'unite': 'compte', 'categorie': 'passes', 'origine': 'FBref', 'signification': 'passes réussies', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': ''}, {'colonne': 'Att_passing', 'unite': 'compte', 'categorie': 'passes', 'origine': 'FBref', 'signification': 'passes tentées', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': ''}, {'colonne': 'Cmp%', 'unite': '%', 'categorie': 'passes', 'origine': 'FBref', 'signification': '% de passes réussies', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': ''}, {'colonne': 'PrgP', 'unite': 'compte', 'categorie': 'progression', 'origine': 'FBref', 'signification': 'passes progressives', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': ''}, {'colonne': 'Fls', 'unite': 'compte', 'categorie': 'discipline', 'origine': 'FBref', 'signification': 'fautes commises', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': ''}, {'colonne': 'Fld', 'unite': 'compte', 'categorie': 'discipline', 'origine': 'FBref', 'signification': 'fautes subies', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': ''}, {'colonne': 'Recov', 'unite': 'compte', 'categorie': 'défense', 'origine': 'FBref', 'signification': 'ballons récupérés', 'utilite_pour_FW': 'travail sans ballon', 'calcul_si_utilisateur': ''}, {'colonne': 'Won', 'unite': 'compte', 'categorie': 'aériens', 'origine': 'FBref', 'signification': 'duels aériens gagnés', 'utilite_pour_FW': 'jeux aériens utiles aux centres', 'calcul_si_utilisateur': ''}, {'colonne': 'Lost_misc', 'unite': 'compte', 'categorie': 'aériens', 'origine': 'FBref', 'signification': 'duels aériens perdus', 'utilite_pour_FW': 'jeux aériens utiles aux centres', 'calcul_si_utilisateur': ''}, {'colonne': 'Won%', 'unite': '%', 'categorie': 'aériens', 'origine': 'FBref', 'signification': '% duels aériens gagnés', 'utilite_pour_FW': 'jeux aériens utiles aux centres', 'calcul_si_utilisateur': ''}, {'colonne': 'Min%', 'unite': '%', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': "% minutes de l'équipe jouées", 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': 'Compl', 'unite': 'compte', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': 'matches complétés (90 min)', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': 'Subs', 'unite': 'compte', 'categorie': 'temps_de_jeu', 'origine': 'FBref', 'signification': 'entrées en jeu', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': ''}, {'colonne': 'PPM', 'unite': 'points/match', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'points par match quand aligné', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': 'onG', 'unite': 'compte', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'buts pour sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': 'onGA', 'unite': 'compte', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'buts contre sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': '+/-', 'unite': 'diff.', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'diff. buts sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': '+/-90', 'unite': 'diff./90', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'diff. buts par 90', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': 'onxG', 'unite': 'xG', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'xG pour sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': 'onxGA', 'unite': 'xG', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'xG contre sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': 'xG+/-', 'unite': 'xG diff.', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'diff. xG sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': 'xG+/-90', 'unite': 'xG diff./90', 'categorie': 'impact', 'origine': 'FBref', 'signification': 'diff. xG par 90', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': ''}, {'colonne': 'Rk_per_90', 'unite': 'par 90', 'categorie': 'meta', 'origine': 'Utilisateur', 'signification': 'rang FBref', 'utilite_pour_FW': 'identification', 'calcul_si_utilisateur': 'Rk_per_90 = Rk / 90s'}, {'colonne': 'CrdY_per_90', 'unite': 'par 90', 'categorie': 'discipline', 'origine': 'Utilisateur', 'signification': 'cartons jaunes', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': 'CrdY_per_90 = CrdY / 90s'}, {'colonne': 'CrdR_per_90', 'unite': 'par 90', 'categorie': 'discipline', 'origine': 'Utilisateur', 'signification': 'cartons rouges', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': 'CrdR_per_90 = CrdR / 90s'}, {'colonne': 'Gls_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'buts', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'Gls_per_90 = Gls / 90s'}, {'colonne': 'Sh_shooting_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'tirs tentés', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'Sh_shooting_per_90 = Sh_shooting / 90s'}, {'colonne': 'SoT_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'tirs cadrés', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'SoT_per_90 = SoT / 90s'}, {'colonne': 'SoT%_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': '% tirs cadrés / tirs', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'SoT%_per_90 = SoT% / 90s'}, {'colonne': 'Sh/90_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'tirs par 90', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'Sh/90_per_90 = Sh/90 / 90s'}, {'colonne': 'SoT/90_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'tirs cadrés par 90', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'SoT/90_per_90 = SoT/90 / 90s'}, {'colonne': 'G/Sh_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'buts par tir', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'G/Sh_per_90 = G/Sh / 90s'}, {'colonne': 'G/SoT_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'buts par tir cadré', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'G/SoT_per_90 = G/SoT / 90s'}, {'colonne': 'Dist_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'distance moyenne de tir (yards)', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'Dist_per_90 = Dist / 90s'}, {'colonne': 'FK_shooting_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'tirs sur CPA directs (CF)', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'FK_shooting_per_90 = FK_shooting / 90s'}, {'colonne': 'xG_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'expected goals', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'xG_per_90 = xG / 90s'}, {'colonne': 'npxG_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'expected goals hors pen.', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'npxG_per_90 = npxG / 90s'}, {'colonne': 'xG_shooting_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': "xG total (table 'Shooting')", 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'xG_shooting_per_90 = xG_shooting / 90s'}, {'colonne': 'npxG_shooting_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': "npxG total (table 'Shooting')", 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'npxG_shooting_per_90 = npxG_shooting / 90s'}, {'colonne': 'npxG/Sh_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'npxG par tir', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'npxG/Sh_per_90 = npxG/Sh / 90s'}, {'colonne': 'G-xG_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'sur/sous-performance vs xG', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'G-xG_per_90 = G-xG / 90s'}, {'colonne': 'np:G-xG_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'sur/sous-performance vs npxG', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'np:G-xG_per_90 = np:G-xG / 90s'}, {'colonne': 'PK_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'buts sur pénalty', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'PK_per_90 = PK / 90s'}, {'colonne': 'PKatt_per_90', 'unite': 'par 90', 'categorie': 'tir', 'origine': 'Utilisateur', 'signification': 'pénaltys tentés', 'utilite_pour_FW': 'volume et efficacité de tir', 'calcul_si_utilisateur': 'PKatt_per_90 = PKatt / 90s'}, {'colonne': 'Ast_per_90', 'unite': 'par 90', 'categorie': 'passes', 'origine': 'Utilisateur', 'signification': 'passes décisives', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': 'Ast_per_90 = Ast / 90s'}, {'colonne': 'KP_per_90', 'unite': 'par 90', 'categorie': 'passes', 'origine': 'Utilisateur', 'signification': 'passes clés (mènent à tir)', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': 'KP_per_90 = KP / 90s'}, {'colonne': 'xAG_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'expected assisted goals', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'xAG_per_90 = xAG / 90s'}, {'colonne': 'xA_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'expected assists', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'xA_per_90 = xA / 90s'}, {'colonne': 'A-xAG_per_90', 'unite': 'par 90', 'categorie': 'expected', 'origine': 'Utilisateur', 'signification': 'sur/sous-perf assists', 'utilite_pour_FW': 'qualité des occasions et durabilité', 'calcul_si_utilisateur': 'A-xAG_per_90 = A-xAG / 90s'}, {'colonne': 'PPA_per_90', 'unite': 'par 90', 'categorie': 'passes', 'origine': 'Utilisateur', 'signification': 'passes complétées vers la surface', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': 'PPA_per_90 = PPA / 90s'}, {'colonne': 'CrsPA_per_90', 'unite': 'par 90', 'categorie': 'passes', 'origine': 'Utilisateur', 'signification': 'centres vers la surface', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': 'CrsPA_per_90 = CrsPA / 90s'}, {'colonne': 'Touches_per_90', 'unite': 'par 90', 'categorie': 'possession', 'origine': 'Utilisateur', 'signification': 'touches de balle', 'utilite_pour_FW': 'présence zones offensives', 'calcul_si_utilisateur': 'Touches_per_90 = Touches / 90s'}, {'colonne': 'Att 3rd_possession_per_90', 'unite': 'par 90', 'categorie': 'possession', 'origine': 'Utilisateur', 'signification': 'touches dans 1/3 offensif', 'utilite_pour_FW': 'présence zones offensives', 'calcul_si_utilisateur': 'Att 3rd_possession_per_90 = Att 3rd_possession / 90s'}, {'colonne': 'Att Pen_per_90', 'unite': 'par 90', 'categorie': 'possession', 'origine': 'Utilisateur', 'signification': 'touches dans la surface adverse', 'utilite_pour_FW': 'présence zones offensives', 'calcul_si_utilisateur': 'Att Pen_per_90 = Att Pen / 90s'}, {'colonne': 'Att_possession_per_90', 'unite': 'par 90', 'categorie': 'dribbles', 'origine': 'Utilisateur', 'signification': 'dribbles tentés (take-ons)', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': 'Att_possession_per_90 = Att_possession / 90s'}, {'colonne': 'Succ_per_90', 'unite': 'par 90', 'categorie': 'dribbles', 'origine': 'Utilisateur', 'signification': 'dribbles réussis', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': 'Succ_per_90 = Succ / 90s'}, {'colonne': 'Succ%_per_90', 'unite': 'par 90', 'categorie': 'dribbles', 'origine': 'Utilisateur', 'signification': '% dribbles réussis', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': 'Succ%_per_90 = Succ% / 90s'}, {'colonne': 'Tkld_per_90', 'unite': 'par 90', 'categorie': 'dribbles', 'origine': 'Utilisateur', 'signification': 'fois taclés en dribble', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': 'Tkld_per_90 = Tkld / 90s'}, {'colonne': 'Tkld%_per_90', 'unite': 'par 90', 'categorie': 'dribbles', 'origine': 'Utilisateur', 'signification': '% de tentatives finies taclé', 'utilite_pour_FW': 'élimination en 1v1', 'calcul_si_utilisateur': 'Tkld%_per_90 = Tkld% / 90s'}, {'colonne': 'Carries_per_90', 'unite': 'par 90', 'categorie': 'conduite', 'origine': 'Utilisateur', 'signification': 'conduites de balle', 'utilite_pour_FW': 'analyse générale', 'calcul_si_utilisateur': 'Carries_per_90 = Carries / 90s'}, {'colonne': 'PrgC_per_90', 'unite': 'par 90', 'categorie': 'progression', 'origine': 'Utilisateur', 'signification': 'conduites progressives', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': 'PrgC_per_90 = PrgC / 90s'}, {'colonne': 'PrgDist_possession_per_90', 'unite': 'par 90', 'categorie': 'progression', 'origine': 'Utilisateur', 'signification': 'distance progressive totale (conduite)', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': 'PrgDist_possession_per_90 = PrgDist_possession / 90s'}, {'colonne': 'CPA_per_90', 'unite': 'par 90', 'categorie': 'progression', 'origine': 'Utilisateur', 'signification': 'conduites dans la surface', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': 'CPA_per_90 = CPA / 90s'}, {'colonne': 'Mis_per_90', 'unite': 'par 90', 'categorie': 'pertes', 'origine': 'Utilisateur', 'signification': 'mauvais contrôles', 'utilite_pour_FW': 'gestion du risque', 'calcul_si_utilisateur': 'Mis_per_90 = Mis / 90s'}, {'colonne': 'Dis_per_90', 'unite': 'par 90', 'categorie': 'pertes', 'origine': 'Utilisateur', 'signification': 'ballons perdus (dépossédé)', 'utilite_pour_FW': 'gestion du risque', 'calcul_si_utilisateur': 'Dis_per_90 = Dis / 90s'}, {'colonne': 'PrgR_per_90', 'unite': 'par 90', 'categorie': 'progression', 'origine': 'Utilisateur', 'signification': 'passes progressives reçues', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': 'PrgR_per_90 = PrgR / 90s'}, {'colonne': 'PrgC_possession_per_90', 'unite': 'par 90', 'categorie': 'progression', 'origine': 'Utilisateur', 'signification': 'conduites progressives (alias)', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': 'PrgC_possession_per_90 = PrgC_possession / 90s'}, {'colonne': '1/3_possession_per_90', 'unite': 'par 90', 'categorie': 'progression', 'origine': 'Utilisateur', 'signification': 'conduites dans le 1/3 offensif', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': '1/3_possession_per_90 = 1/3_possession / 90s'}, {'colonne': 'Cmp_per_90', 'unite': 'par 90', 'categorie': 'passes', 'origine': 'Utilisateur', 'signification': 'passes réussies', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': 'Cmp_per_90 = Cmp / 90s'}, {'colonne': 'Att_passing_per_90', 'unite': 'par 90', 'categorie': 'passes', 'origine': 'Utilisateur', 'signification': 'passes tentées', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': 'Att_passing_per_90 = Att_passing / 90s'}, {'colonne': 'Cmp%_per_90', 'unite': 'par 90', 'categorie': 'passes', 'origine': 'Utilisateur', 'signification': '% de passes réussies', 'utilite_pour_FW': "création d'occasions", 'calcul_si_utilisateur': 'Cmp%_per_90 = Cmp% / 90s'}, {'colonne': 'PrgP_per_90', 'unite': 'par 90', 'categorie': 'progression', 'origine': 'Utilisateur', 'signification': 'passes progressives', 'utilite_pour_FW': 'capacité à faire progresser le jeu', 'calcul_si_utilisateur': 'PrgP_per_90 = PrgP / 90s'}, {'colonne': 'Fls_per_90', 'unite': 'par 90', 'categorie': 'discipline', 'origine': 'Utilisateur', 'signification': 'fautes commises', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': 'Fls_per_90 = Fls / 90s'}, {'colonne': 'Fld_per_90', 'unite': 'par 90', 'categorie': 'discipline', 'origine': 'Utilisateur', 'signification': 'fautes subies', 'utilite_pour_FW': 'risque de suspension', 'calcul_si_utilisateur': 'Fld_per_90 = Fld / 90s'}, {'colonne': 'Recov_per_90', 'unite': 'par 90', 'categorie': 'défense', 'origine': 'Utilisateur', 'signification': 'ballons récupérés', 'utilite_pour_FW': 'travail sans ballon', 'calcul_si_utilisateur': 'Recov_per_90 = Recov / 90s'}, {'colonne': 'Won_per_90', 'unite': 'par 90', 'categorie': 'aériens', 'origine': 'Utilisateur', 'signification': 'duels aériens gagnés', 'utilite_pour_FW': 'jeux aériens utiles aux centres', 'calcul_si_utilisateur': 'Won_per_90 = Won / 90s'}, {'colonne': 'Lost_misc_per_90', 'unite': 'par 90', 'categorie': 'aériens', 'origine': 'Utilisateur', 'signification': 'duels aériens perdus', 'utilite_pour_FW': 'jeux aériens utiles aux centres', 'calcul_si_utilisateur': 'Lost_misc_per_90 = Lost_misc / 90s'}, {'colonne': 'Won%_per_90', 'unite': 'par 90', 'categorie': 'aériens', 'origine': 'Utilisateur', 'signification': '% duels aériens gagnés', 'utilite_pour_FW': 'jeux aériens utiles aux centres', 'calcul_si_utilisateur': 'Won%_per_90 = Won% / 90s'}, {'colonne': 'Min%_per_90', 'unite': 'par 90', 'categorie': 'temps_de_jeu', 'origine': 'Utilisateur', 'signification': "% minutes de l'équipe jouées", 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': 'Min%_per_90 = Min% / 90s'}, {'colonne': 'Compl_per_90', 'unite': 'par 90', 'categorie': 'temps_de_jeu', 'origine': 'Utilisateur', 'signification': 'matches complétés (90 min)', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': 'Compl_per_90 = Compl / 90s'}, {'colonne': 'Subs_per_90', 'unite': 'par 90', 'categorie': 'temps_de_jeu', 'origine': 'Utilisateur', 'signification': 'entrées en jeu', 'utilite_pour_FW': 'disponibilité et rôle', 'calcul_si_utilisateur': 'Subs_per_90 = Subs / 90s'}, {'colonne': 'PPM_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'points par match quand aligné', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': 'PPM_per_90 = PPM / 90s'}, {'colonne': 'onG_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'buts pour sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': 'onG_per_90 = onG / 90s'}, {'colonne': 'onGA_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'buts contre sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': 'onGA_per_90 = onGA / 90s'}, {'colonne': '+/-_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'diff. buts sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': '+/-_per_90 = +/- / 90s'}, {'colonne': '+/-90_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'diff. buts par 90', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': '+/-90_per_90 = +/-90 / 90s'}, {'colonne': 'onxG_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'xG pour sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': 'onxG_per_90 = onxG / 90s'}, {'colonne': 'onxGA_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'xG contre sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': 'onxGA_per_90 = onxGA / 90s'}, {'colonne': 'xG+/-_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'diff. xG sur le terrain', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': 'xG+/-_per_90 = xG+/- / 90s'}, {'colonne': 'xG+/-90_per_90', 'unite': 'par 90', 'categorie': 'impact', 'origine': 'Utilisateur', 'signification': 'diff. xG par 90', 'utilite_pour_FW': 'impact quand sur le terrain', 'calcul_si_utilisateur': 'xG+/-90_per_90 = xG+/-90 / 90s'}]
col_dict = pd.DataFrame(_dict)
col_dict.sort_values("colonne", inplace=True)
col_dict.reset_index(drop=True, inplace=True)
col_dict.head(10)


,colonne,unite,categorie,origine,signification,utilite_pour_FW,calcul_si_utilisateur
0,+/-,diff.,impact,FBref,diff. buts sur le terrain,impact quand sur le terrain,
1,+/-90,diff./90,impact,FBref,diff. buts par 90,impact quand sur le terrain,
2,+/-90_per_90,par 90,impact,Utilisateur,diff. buts par 90,impact quand sur le terrain,+/-90_per_90 = +/-90 / 90s
3,+/-_per_90,par 90,impact,Utilisateur,diff. buts sur le terrain,impact quand sur le terrain,+/-_per_90 = +/- / 90s
4,1/3_possession,compte,progression,FBref,conduites dans le 1/3 offensif,capacité à faire progresser le jeu,
5,1/3_possession_per_90,par 90,progression,Utilisateur,conduites dans le 1/3 offensif,capacité à faire progresser le jeu,1/3_possession_per_90 = 1/3_possession / 90s
6,90s,matchs de 90 min,temps_de_jeu,FBref,équivalents 90 minutes,disponibilité et rôle,
7,A-xAG,Ast - xAG,expected,FBref,sur/sous-perf assists,qualité des occasions et durabilité,
8,A-xAG_per_90,par 90,expected,Utilisateur,sur/sous-perf assists,qualité des occasions et durabilité,A-xAG_per_90 = A-xAG / 90s
9,Age,années,temps_de_jeu,FBref,âge saison,disponibilité et rôle,


In [7]:
kpi_df = df.copy()
kpi_df['NP_Gls_per90'] = safe_div(kpi_df['Gls'] - kpi_df['PK'], kpi_df['90s'])
kpi_df['npxG_per90'] = kpi_df['npxG'] / kpi_df['90s']
kpi_df['Shots_per90'] = kpi_df['Sh/90']
kpi_df['Shot_OnTarget_pct'] = kpi_df['SoT%']
kpi_df['Conv_rate'] = kpi_df['G/Sh']
kpi_df['xG_per_shot_np'] = kpi_df['npxG/Sh']
kpi_df['Finishing_vs_xG_np_per90'] = safe_div((kpi_df['Gls'] - kpi_df['PK'] - kpi_df['npxG']), kpi_df['90s'])
kpi_df['xGxA_per90'] = (kpi_df['xG'] + kpi_df['xA']) / kpi_df['90s']
kpi_df['GA_per90'] = (kpi_df['Gls'] + kpi_df['Ast']) / kpi_df['90s']
kpi_df['KeyPasses_per90'] = kpi_df['KP_per_90'] if 'KP_per_90' in kpi_df else kpi_df['KP'] / kpi_df['90s']
kpi_df['PPA_per90'] = kpi_df['PPA_per_90'] if 'PPA_per_90' in kpi_df else kpi_df['PPA'] / kpi_df['90s']
kpi_df['PrgPass_per90'] = kpi_df['PrgP_per_90'] if 'PrgP_per_90' in kpi_df else kpi_df['PrgP'] / kpi_df['90s']
kpi_df['PrgCarries_per90'] = kpi_df['PrgC_per_90'] if 'PrgC_per_90' in kpi_df else kpi_df['PrgC'] / kpi_df['90s']
kpi_df['Touches_Att3rd_per90'] = kpi_df['Att 3rd_possession_per_90'] if 'Att 3rd_possession_per_90' in kpi_df else kpi_df['Att 3rd_possession'] / kpi_df['90s']
kpi_df['Touches_Box_per90'] = kpi_df['Att Pen_per_90'] if 'Att Pen_per_90' in kpi_df else kpi_df['Att Pen'] / kpi_df['90s']
kpi_df['Dribble_Success_pct'] = kpi_df['Succ%']
kpi_df['Dribble_Att_per90'] = kpi_df['Att_possession_per_90'] if 'Att_possession_per_90' in kpi_df else kpi_df['Att_possession'] / kpi_df['90s']
kpi_df['Fouls_Drawn_per90'] = kpi_df['Fld_per_90'] if 'Fld_per_90' in kpi_df else kpi_df['Fld'] / kpi_df['90s']
kpi_df['Recoveries_per90'] = kpi_df['Recov_per_90'] if 'Recov_per_90' in kpi_df else kpi_df['Recov'] / kpi_df['90s']
kpi_df['Aerial_Win_pct'] = kpi_df['Won%']
kpi_df['OnField_xGD_per90'] = kpi_df['xG+/-90']
kpi_df['Availability_Min_pct'] = kpi_df['Min%']
kpi_cols = ['NP_Gls_per90', 'npxG_per90', 'Shots_per90', 'Shot_OnTarget_pct', 'Conv_rate', 'xG_per_shot_np', 'Finishing_vs_xG_np_per90', 'xGxA_per90', 'GA_per90', 'KeyPasses_per90', 'PPA_per90', 'PrgPass_per90', 'PrgCarries_per90', 'Touches_Att3rd_per90', 'Touches_Box_per90', 'Dribble_Success_pct', 'Dribble_Att_per90', 'Fouls_Drawn_per90', 'Recoveries_per90', 'Aerial_Win_pct', 'OnField_xGD_per90', 'Availability_Min_pct']
kpi_view = kpi_df[['Player','Squad','Age','MainPos','Min','90s'] + kpi_cols].copy()
kpi_view.head(10)

,Player,Squad,Age,MainPos,Min,90s,NP_Gls_per90,npxG_per90,Shots_per90,Shot_OnTarget_pct,...,PrgCarries_per90,Touches_Att3rd_per90,Touches_Box_per90,Dribble_Success_pct,Dribble_Att_per90,Fouls_Drawn_per90,Recoveries_per90,Aerial_Win_pct,OnField_xGD_per90,Availability_Min_pct
0,Aaron Seydel,Darmstadt 98,27.0,FW,619,6.9,0.144928,0.144928,2.04,42.9,...,1.017771,18.610662,4.507270,27.3,1.599354,1.017771,2.035541,59.20,-1.14,20.20
1,Abde Ezzalzouli,Betis,21.0,FW,933,9.6,0.104167,0.229167,4.36,24.6,...,5.498392,26.141479,5.787781,46.0,4.823151,1.543408,4.244373,41.65,-0.13,13.65
2,Abde Rebbach,Alavés,24.0,FW,722,8.0,0.125000,0.100000,2.12,23.5,...,4.238227,24.681440,4.487535,35.7,3.490305,1.246537,3.864266,35.00,0.09,21.10
3,Abdoul Kader Bamba,Nantes,29.0,FW,611,6.8,0.294118,0.308824,2.06,35.7,...,6.039280,30.049100,3.829787,32.3,4.566285,5.155483,6.481178,50.00,-0.89,20.00
4,Abdoulaye Doucouré,Everton,30.0,FW,2629,29.2,0.239726,0.301370,1.61,44.7,...,1.882845,16.945607,3.423355,52.1,1.643210,0.273868,4.929631,37.50,-0.05,76.90
5,Abdón Prats,Mallorca,30.0,FW,1270,14.1,0.425532,0.226950,2.34,42.4,...,0.637795,12.826772,3.047244,44.4,1.275591,1.275591,2.125984,39.40,0.08,37.10
6,Adam Ounas,Lille,26.0,FW,418,4.6,0.217391,0.130435,1.94,33.3,...,4.306220,28.636364,3.229665,64.0,5.382775,2.583732,5.813397,50.00,0.90,13.70
7,Adama Traoré,Fulham,27.0,FW,377,4.2,0.476190,0.357143,2.63,63.6,...,6.923077,28.169761,5.729443,52.9,4.058355,0.954907,3.103448,60.00,-1.21,11.00
8,Ademola Lookman,Atalanta,25.0,FW,1894,21.0,0.523810,0.433333,3.18,35.8,...,3.421331,28.843717,6.129884,44.8,4.134108,1.615628,4.229145,46.20,0.31,55.40
9,Adnan Januzaj,Sevilla,28.0,FW,193,2.1,0.000000,0.285714,2.80,16.7,...,0.466321,26.580311,2.331606,45.5,5.129534,4.196891,4.196891,75.00,-1.00,5.60


In [9]:

# Sauvegarde du tableau KPI
kpi_view.to_csv(r"kpi_fw.csv", index=False)
r"kpi_fw.csv"


'kpi_fw.csv'